<a href="https://colab.research.google.com/github/Swathi1309/ED18B034_ME18B133_Assignment1/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb
!wandb login

In [2]:
import keras
from keras.datasets import fashion_mnist
from PIL import Image
import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

wandb.init(project="assignment-1", entity="swathi")
classes = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

wandb: Currently logged in as: swathi (use `wandb login --relogin` to force relogin)


In [3]:
def load_images():
  (x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()
  X_train_img = x_train[:54000]
  X_validation_img = x_train[54000:]
  X_test_img = x_test
  return X_train_img, X_validation_img, X_test_img

In [4]:
def load_data():
  (x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()
  X_train = np.transpose((np.array(x_train[:54000],dtype=int).reshape([54000,784]))/4)
  Y_train = (np.array(y_train[:54000],dtype=int).reshape([54000,1]))
  X_validation = np.transpose((np.array(x_train[54000:],dtype=int).reshape([6000,784]))/4)
  Y_validation = np.array(y_train[54000:],dtype=int).reshape([6000,1])
  X_test = np.transpose((np.array(x_test,dtype=int).reshape([10000,784]))/4)
  Y_test = np.array(y_test,dtype=int).reshape([10000,1])
  return X_train, Y_train, X_validation, Y_validation, X_test, Y_test

In [17]:
def log_images():
  images = []
  labels = []
  for i in range(len(X_train_img)):
    if len(labels)==len(classes):
      break
    j = random.randint(0,53999)
    if classes[Y_train[j][0]] not in labels:
      images.append(X_train_img[j])
      labels.append(classes[Y_train[j][0]])
  wandb.log({"Examples": [wandb.Image(img, caption=caption) for img,caption in zip(images,labels)]})

Loading data

In [22]:
X_train, Y_train, X_validation, Y_validation, X_test, Y_test = load_data()
X_input = X_train
X_train_img, X_validation_img, X_test_img = load_images()

In [20]:
log_images()

Defining basic parameters

In [23]:
N= [0,3,10]  #Insert the number of neurons in each layer
N[0]=X_input.shape[0]
l = len(N) - 1
np.random.seed(0)

Weights initialising

In [24]:
def initialize_weights(N):
  W = []
  b = []
  W.append(0)
  b.append(0)
  for i in range(1,l+1):
    W.append(np.random.randn(N[i],N[i-1]))
    b.append(np.random.randn(N[i],1))

  return W,b
  

In [26]:
def one_hot(y):
  one_hot_y=np.zeros((10,X_input.shape[1]))
  for i in range(N[1]):
    one_hot_y[y[:,i],i] = 1
  
  return one_hot_y

Softmax

In [25]:
def softmax(a):
  return np.exp(a)/sum(np.exp(a))

Sigmoid

In [27]:
def sigmoid(a):

  return 1/(1+np.exp(-a))


Sigmoid derivative

In [28]:
def dSigmoid(a):

  return sigmoid(a)*(1-sigmoid(a))

Feed forward network

In [29]:
def feedforward(X_input, W, b, N, l):
  a=[]
  a.append(0)
  h=[]
  h.append(X_input)
  for i in range(1,l):
    # print(i)
    a_new=b[i]+np.matmul(W[i],h[i-1])
    h_new=sigmoid(a_new)
    a.append(a_new)
    h.append(h_new)

  a_new=b[l]+np.matmul(W[l],h[l-1])
  y_hat=softmax(a_new)
  a.append(a_new)

  return a,h,y_hat

Backpropogation

In [30]:
def backrop(y,y_hat,W,a,h):
  grad_W=[]
  grad_b=[]
  grad_a=[]
  grad_h=[]
  
  da = y_hat - one_hot(y)
  grad_a.append(da)

  for i in reversed(range(1,l+1)):
    dw = np.matmul(da,np.transpose(h[i-1]))
    db = da
    dh_prev = np.matmul(np.transpose(W[i]),da)
    da_prev = dh_prev*dSigmoid(a[i-1])
    
    grad_W.append(dw)
    grad_b.append(db)
    grad_a.append(da_prev)
    grad_h.append(dh_prev)
    da=da_prev

  grad_W.append(0)
  grad_b.append(0)
  grad_W.reverse()
  grad_b.reverse()
  grad_a.reverse()
  grad_h.reverse()
  

  return grad_W,grad_b,grad_a,grad_h


Gradient Descent

In [31]:
def grad_descent(W,b,grad_W,grad_b):
  W_new=[]
  W_new.append(0)
  b_new=[]
  b_new.append(0)
  for i in range(1,l+1):
    W_new.append(W[i] - grad_W[i])
    b_new.append(b[i] - grad_b[i])
  
  return W_new,b_new


Training

In [32]:
def training(N,l,epochs,X_input,y):
  
  W , b = initialize_weights(N)
  for j in range(epochs):
    a , h , y_hat = feedforward(X_input, W, b, N, l)  
    grad_W , grad_b = backrop(y,y_hat,W,a,h)[0:2]
    # print(len(W),len(b),len(grad_W),len(grad_b))
    W , b = grad_descent(W,b,grad_W,grad_b)

  return W,b


In [33]:
def predict(W,b,X_test,N,l):
  y_hat = feedforward(X_test, W, b, N, l)[2]
  y_class = np.argmax(y_hat,axis=0)

  return y_class

In [34]:
def calculate_loss(y_hat,Y):
  loss = 0
  for i in range(Y.shape[1]):
    loss += -math.log10(y_hat[Y[0][i]][i])
  print (loss)

In [35]:
def gradient_descent(epochs, learn_rate):
  W, b = initialize_weights()
  a, h, y_hat = feed_forward(W,b)
  print (y_hat)
  calculate_loss(y_hat, Y)
  W_new = W
  b_new = b
  for i in range(epochs):
    a, h, y_hat = feed_forward(W,b)
    grad_W, grad_b, grad_h, grad_a = backrop(Y, y_hat, W, a, h)
    for j in range(1,L+1):
      W_new[j] = W[j] - grad_W[j]
      b_new[j] = b[j] - grad_b[j]
    W = W_new
    b = b_new
  a, h, y_hat = feed_forward(W,b)
  calculate_loss(y_hat, Y)
  print (y_hat)

In [ ]:
w_new,b_new= training(N,l,3,X_input,Y_train)